In [ ]:
from huggingface_hub import login
from getpass import getpass

login(token=getpass("🔑 Enter your HF write token: "))

🔑 Enter your HF write token: ··········


In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.10.1", "triton==3.2.0") if is_t4 else ("vllm", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers transformers
    !uv pip install -qqq {get_triton}

In [ ]:
import re
import numpy as np
import torch

_ANSWER_RE = re.compile(
    r"<answer>\s*(.*?)\s*</answer>", flags=re.DOTALL | re.IGNORECASE
)
_USER_SPAN_RE = re.compile(
    r"<\|user\|>\s*(.*?)\s*</s>", flags=re.DOTALL | re.IGNORECASE
)


def _extract_text_between(s: str, pattern: re.Pattern, fallback: str = "") -> str:
    m = pattern.search(s or "")
    return m.group(1).strip() if m else (fallback or "").strip()


def _extract_utterance_from_prompt(prompt_text: str) -> str:
    """Pull just the user utterance from your chat template."""
    text = _extract_text_between(prompt_text, _USER_SPAN_RE, fallback=prompt_text)
    # strip any residual tags
    return re.sub(r"</?[^>]+>", "", text).strip()


def _extract_answer_text(reply_text: str) -> str:
    """If XML is present, score only the <answer>…</answer> body."""
    ans = _extract_text_between(reply_text or "", _ANSWER_RE, fallback=reply_text or "")
    return ans.strip()


def _flatten_completions(completions) -> list[str]:
    """Handle various TRL completion shapes and return list[str]."""
    out = []
    for c in completions or []:
        if isinstance(c, str):
            out.append(c)
        elif isinstance(c, dict) and "content" in c:
            out.append(c["content"])
        elif isinstance(c, (list, tuple)) and len(c) > 0:
            first = c[0]
            if isinstance(first, dict) and "content" in first:
                out.append(first["content"])
            elif (
                isinstance(first, (list, tuple))
                and len(first) > 0
                and isinstance(first[0], dict)
                and "content" in first[0]
            ):
                out.append(first[0]["content"])
            else:
                out.append(str(c))
        else:
            out.append("")
    return out


def _batch_calibrate(raw_scores: np.ndarray, temperature: float = 0.6) -> np.ndarray:
    raw_scores = np.asarray(raw_scores, dtype=float)
    if raw_scores.size == 0:
        return raw_scores
    raw_scores = np.nan_to_num(raw_scores, nan=0.0, posinf=1.0, neginf=0.0)
    mu, sigma = raw_scores.mean(), raw_scores.std()
    if sigma < 1e-6:
        # avoid divide-by-zero; simple sigmoid on centered scores
        return np.clip(1.0 / (1.0 + np.exp(-(raw_scores - mu))), 0.0, 1.0)
    z = (raw_scores - mu) / sigma
    t = max(1e-4, float(temperature))
    return 1.0 / (1.0 + np.exp(-z / t))


from sentence_transformers import CrossEncoder

_ce = CrossEncoder(
    "cross-encoder/stsb-roberta-large",
    device="cuda" if torch.cuda.is_available() else "cpu",
)


def semantic_sts_reward(prompts, completions, **kwargs) -> list[float]:
    """
    Reward = calibrated semantic similarity between:
      source = user utterance extracted from the prompt
      reply  = model's <answer> text (or full reply if no XML)
    Returns floats in [0,1].
    """
    user_msgs = [p[-1]["content"] for p in prompts]  # last msg is your user scaffold
    sources = [_extract_utterance_from_prompt(m) for m in user_msgs]

    reply_texts = _flatten_completions(completions)
    replies = [_extract_answer_text(t) for t in reply_texts]

    pairs = []
    for s, r in zip(sources, replies):
        s = (s or "").strip()
        r = (r or "").strip()
        pairs.append((s if s else "x", r if r else "x"))

    try:
        raw = np.array(_ce.predict(pairs, batch_size=64), dtype=float)
    except Exception:
        raw = np.zeros(len(pairs), dtype=float)

    raw = np.nan_to_num(raw, nan=0.0, posinf=1.0, neginf=0.0)
    if raw.size and raw.max() > 1.25:
        raw = raw / 5.0
    raw = np.clip(raw, 0.0, 1.0)

    cal = _batch_calibrate(raw, temperature=0.6)
    return cal.tolist()


from transformers import AutoTokenizer, AutoModelForSequenceClassification

_EMPATHY_REPO = "miladsolo/roberta-lora-wassa-empathy"
_device = "cuda" if torch.cuda.is_available() else "cpu"
_tok = AutoTokenizer.from_pretrained(_EMPATHY_REPO)
_cls = AutoModelForSequenceClassification.from_pretrained(_EMPATHY_REPO)
_cls.eval().to(_device)


def predict(texts, max_len=256):
    enc = _tok(
        texts, padding=True, truncation=True, max_length=max_len, return_tensors="pt"
    )
    enc = {k: v.to(_device) for k, v in enc.items()}
    with torch.no_grad():
        logits = _cls(**enc).logits  # [B, 3] -> [Emotion, EmotionalPolarity, Empathy]
    arr = logits.detach().cpu().numpy()
    return [
        {
            "Emotion": float(a[0]),
            "EmotionalPolarity": float(a[1]),
            "Empathy": float(a[2]),
        }
        for a in arr
    ]


def empathy_model_reward(prompts=None, completions=None, **kwargs) -> list[float]:
    """
    Reward = model-predicted Empathy logit for the assistant's reply (higher is better).
    Uses miladsolo/roberta-lora-wassa-empathy via `predict()`. Calibrated to [0,1].
    """
    reply_texts = _flatten_completions(completions or [])
    answers = [_extract_answer_text(t) for t in reply_texts]
    safe_inputs = [a if a else " " for a in answers]

    preds = predict(safe_inputs)
    raw = np.array([p.get("Empathy", 0.0) for p in preds], dtype=float)

    cal = _batch_calibrate(raw, temperature=0.6)
    return np.clip(cal, 0.0, 1.0).tolist()


In [ ]:
import random
import re
from typing import Optional, Sequence

from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from unsloth import FastLanguageModel
from trl import GRPOConfig, GRPOTrainer

import torch

max_seq_length = 1024  
lora_rank = 32         

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,     
    fast_inference = True,   
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6,  
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], 
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",  
    random_state = 3407,  
)

SYSTEM_PROMPT = """
<|system|>
You are a friendly, trauma-informed assistant. Analyze the user's message carefully for emotional content, intensity, and empathy needs.

When responding:
1. First, analyze what the user is expressing (concern, emotion, intensity level 0-5)
2. Then provide an empathetic response that matches their emotional needs
3. Your response should reflect their experience, validate feelings, and optionally ask ONE gentle question
4. Keep responses to 1-2 sentences, avoid emojis, lists, quotes, or clinical tone

Output EXACTLY this XML format:

<reasoning>
- User's main concern: [identify what they're expressing]
- Emotional content: [what emotions/feelings are present]
- Intensity level: [0=neutral, 1-2=mild, 3-4=moderate, 5=high emotional intensity]
- Response approach: [how to respond empathetically]
</reasoning>
<answer>
[Your empathetic response here - 1-2 sentences maximum]
</answer>
</s>
""".strip()


def _mk_instruction(utterance: str) -> str:
    """Format user text into Llama chat template structure."""
    return (
        "Here is the dialogue so far. Continue as <|assistant|>.\n\n"
        f"<|user|>\n{utterance}\n</s>\n"
    )

def load_wassa_empathy(split: Optional[str] = None) -> Dataset:
    """
    Load and format WASSA empathy dataset for GRPO training.
    Updated to match current repository implementation.
    """
    # Load dataset - use train split by default  
    if split is None:
        ds = load_dataset("miladsolo/wassa-conv-turn-empathy", split="train")
    else:
        ds = load_dataset("miladsolo/wassa-conv-turn-empathy", split=split)

    def _map(ex):
        # Get text from "text" field (confirmed from repository analysis)
        text = (ex.get("text") or "").strip()
        if not text:
            return {"prompt": None}
            
        # Format user message for Llama chat template
        user_msg = _mk_instruction(text)
        
        return {
            "prompt": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_msg},
            ],
        }

    # Apply mapping and filter out invalid entries
    ds = ds.map(_map)
    ds = ds.filter(lambda ex: ex["prompt"] is not None)

    print(f"✅ Prepared {len(ds)} WASSA empathy examples for GRPO training.")
    print("📝 Dataset structure: Each example contains prompt for model to generate empathetic response")
    print("🏆 Reward functions will score generated responses for empathy quality")

    # Quick preview
    for i in range(min(3, len(ds))):
        ex = ds[i]
        print(f"\n--- Sample {i} ---")
        user_content = ex["prompt"][-1]["content"]
        # Extract just the user message for cleaner display  
        user_text = user_content.split("<|user|>")[-1].split("</s>")[0].strip()
        print(f"📱 User Input: '{user_text}'")
        print("🤖 Model Task: Analyze emotion → Generate reasoning + empathetic response")

    return ds

dataset = load_wassa_empathy()

max_prompt_length = 512  

training_args = GRPOConfig(
    learning_rate = 1e-3,  
    adam_beta1 = 0.9,      
    adam_beta2 = 0.99,     
    weight_decay = 0.1,    
    warmup_ratio = 0.1,    
    lr_scheduler_type = "cosine",  
    optim = "paged_adamw_8bit",    
    logging_steps = 1,     
    per_device_train_batch_size = 1,  
    gradient_accumulation_steps = 1,   
    num_generations = 3,   
    max_prompt_length = max_prompt_length,  
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 250,       
    save_steps = 250,        
    max_grad_norm = 0.1,   
    report_to = "tensorboard", 
    output_dir = "outputs",     
)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        semantic_sts_reward,    # Use real empathy rewards instead of random
        empathy_model_reward,
    ],
    args = training_args,
    train_dataset = dataset,
)

print("🚀 Starting GRPO training with empathy rewards...")
trainer.train()

print("\n=== Testing Inference ===")

test_messages = [
    "I'm feeling really overwhelmed with work lately.",
    "It's definitely really sad to read, considering everything they're all going through.",
    "I think it's super sad... they seem to never catch a break, always struggling."
]

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,      
    top_p = 0.95,
    max_tokens = 1024,
)

# Test each message
for i, test_message in enumerate(test_messages):
    print(f"\n--- Test {i+1}: {test_message} ---")
    
    # Build prompt WITH the SYSTEM_PROMPT for empathy
    test_prompt = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Here is the dialogue so far. Continue as <|assistant|>.\n\n<|user|>\n{test_message}\n</s>\n"}
        ],
        tokenize = False,
        add_generation_prompt = True,
    )
    
    # Generate with trained model
    output = model.fast_generate(
        [test_prompt],
        sampling_params = sampling_params,
        lora_request = None,  # Using the trained model directly
    )[0].outputs[0].text
    
    print("Response:")
    print(output[:500] + ("..." if len(output) > 500 else ""))
    print("-" * 50)

# Save LoRA
print("\n💾 Saving trained LoRA adapter...")
model.save_lora("grpo_saved_lora")
print("✅ LoRA adapter saved to: grpo_saved_lora")